In [13]:
# Trazer as opções de configuração

import json

with open('./options.json', 'r') as file:
    data = json.load(file)
file.close()

In [14]:
# Manejo da data dos investimentos

from datetime import date

def transform_date_string_on_number_list(date_str):
    date_list = date_str.split('-')

    for index, element in enumerate(date_list):
        date_list[index] = int(date_list[index])
    
    return date_list

START_DATE = data["CONFIG"]["START_DATE"]
TODAY_DATE = str(date.today())
start_year, start_month, start_day = transform_date_string_on_number_list(START_DATE)
today_year, today_month, today_day = transform_date_string_on_number_list(TODAY_DATE)


In [15]:
# Criar pastas para os arquivos

import os 

dir = os.path.join(os.path.abspath('../'), 'archives')
dir_zip = os.path.join(os.path.abspath(dir), data['CONFIG']['DIR_ZIP_NAME'])
dir_csv = os.path.join(os.path.abspath(dir), data['CONFIG']['DIR_CSV_NAME'])

if not os.path.exists(dir):
    os.mkdir(dir)

if not os.path.exists(dir_zip):
    os.mkdir(dir_zip)

if not os.path.exists(dir_csv):
    os.mkdir(dir_csv)

In [16]:
# Realizar download dos arquivos da CVM

from urllib.request import urlretrieve, HTTPError

def adjust_month (month):
    if month < 10:
        month = '0' + str(month)
    
    return str(month)

def download_inf_diario_archives (year, month):
    year = str(year)
    month = str(month)

    link = 'https://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_' + year + month + '.zip'
    urlretrieve(link, '../archives/archives-zip/' + year + month + '.zip')

for year in range (start_year, today_year + 1):
    
    FIRST_MONTH_YEAR, LAST_MONTH_YEAR = 1, 12
    start_month_loop, end_month_loop = '', ''
    exception_case = False

    if start_year == year:
        start_month_loop = start_month
        end_month_loop = LAST_MONTH_YEAR
        exception_case = True

    if today_year == year:
        start_month_loop = FIRST_MONTH_YEAR
        end_month_loop = today_month
        exception_case = True

    if exception_case == False:
        start_month_loop = FIRST_MONTH_YEAR
        end_month_loop = LAST_MONTH_YEAR

    for month in range (start_month_loop, end_month_loop + 1):
        month = adjust_month(month)

        try:
            download_inf_diario_archives(year, month)
        except HTTPError as err: 
            if err.code == 404:
                print('Link não encontrado, ainda não foi disponibilizado o arquivo do mês ' + month + 'e do ano ' + year)
            else:
                print(err.msg)

In [17]:
# Unzip arquivos

from shutil import unpack_archive

archives_list = os.listdir(dir_zip)

ARCHIVE_TEMPLATE = 'inf_diario_fi_'
ARCHIVE_EXTENSION = '.csv'

for archive_zip_name in archives_list:
    archive_date = archive_zip_name[0:6]
    archive_csv_name = ARCHIVE_TEMPLATE + archive_date + ARCHIVE_EXTENSION

    archive_csv_search = os.path.join(dir_csv, archive_csv_name)
    archive_zip_search = os.path.join(dir_zip, archive_zip_name)

    if not os.path.exists(archive_csv_search):
        unpack_archive(archive_zip_search, dir_csv)
